# 첫번째 시도

In [ ]:
import heapq
q = []
N, K = map(int, input().split())
for table in range(1, K+1):
    num, w = map(int, input().split())
    heapq.heappush(q, [w, -table, num])

next_person = K + 1
result = 0
mul_num = 1
while q:
    # 가장 빨린 끝난 사람 종료
    end_w, end_table, end_num = heapq.heappop(q)
    # 정답 갱신
    result += mul_num * end_num
    mul_num += 1
    # 남은 시간
    for i in range(len(q)):
        q[i][0] -= end_w
    if next_person < N+1:
        # 새로운 사람 등장
        num, w = map(int, input().split())
        heapq.heappush(q, [w, end_table, num])    
    # 다음 사람
    next_person += 1
print(result)

[문제]
- 시간 초과
- 13900이 나와야 하는데, 13942가 나옴.<br>

[시간복잡도]<br>
O(N*K)

# 피드백
## 1. 로직 / 정확도 관점에서 문제점
1. N < K일 때 입력 초과
```py
for table in range(1, K+1):
    num, w = map(int, input().split())
```
N이 K보다 작으면, 존재하지 않는 줄을 읽으려고 해서 바로 런타임 에러 터짐.
→ range(1, min(N, K)+1) 로 해야 돼.

2. “가장 적게 기다리는 계산대” 규칙이 제대로 구현 안 됨

지금 로직은:
- 처음 K명은 계산대 1~K에 한 명씩.
- 이후엔 항상 방금 끝난 계산대에만 다음 사람을 붙임.

하지만 문제는 “현재까지의 예상 대기시간이 가장 짧은 계산대”를 골라야 하니까,
실제로는 K개의 계산대 각각의 누적 작업량(언제 비는지) 를 관리하면서
항상 그중 최소인 계산대에 배정해야 해.

지금 코드는 “끝나는 순서대로만 붙이는” 방식이라
원래 문제의 시뮬레이션이랑은 좀 다름. -> 동시에 끝나면 번호 작은곳에 넣어야 하는데, 지금은 그냥 번호 큰곳에 넣고 있음

3. 시간을 계속 깎는 방식 자체가 비효율적
한 번 pop할 때마다 for i in range(len(q)): 로
모든 사람의 남은 시간을 줄이는 설계가 문제의 원인이야.

사실 이건 안 해도 되고, “절대 시각(완료 시간)” 만 관리하면 돼.

## 2. 어떻게 바꿔야 하는지 (권장 방식)
### 핵심 아이디어

- 계산대 힙: (현재까지의 누적 시간, 계산대 번호)
→ 새 손님을 항상 이 힙의 top 계산대에 배정

- 동시에, 각 손님의 퇴장 정보를 (퇴장시간, -계산대번호, 회원번호) 형태로 배열에 모아서
마지막에 정렬한 뒤, 1×r₁ + 2×r₂ + … 계산.

이렇게 하면 매 손님당
- 힙 pop + push → O(log K)

이므로 전체:
O(N log K) (＋ 마지막 정렬 O(N log N))
정도로 깔끔하게 떨어져.

In [ ]:
import heapq
import sys
input = sys.stdin.readline

N, K = map(int, input().split())
customers = [tuple(map(int, input().split())) for _ in range(N)]  # (id, w)

pq = []           # (finish_time, counter_idx, customer_id)
exit_list = []    # (finish_time, -counter_idx, id)

cnt = min(N, K)

# 처음 K명 계산대 배치
for i in range(cnt):
    cid, w = customers[i]
    heapq.heappush(pq, (w, i+1, cid))  # 계산대 1~K

next_idx = cnt  # 다음 고객 index

# K명 이후 고객 처리
while next_idx < N:
    finish_time, counter, cid = heapq.heappop(pq)
    exit_list.append((finish_time, -counter, cid))

    new_id, new_w = customers[next_idx]
    heapq.heappush(pq, (finish_time + new_w, counter, new_id))
    next_idx += 1

# 계산대에 남은 고객 처리
while pq:
    finish_time, counter, cid = heapq.heappop(pq)
    exit_list.append((finish_time, -counter, cid))

# 퇴장 규칙 정렬
exit_list.sort()

# 정답 계산
ans = 0
for i, (_, _, cid) in enumerate(exit_list, start=1):
    ans += i * cid

print(ans)
